In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import pytorch_lightning as pl
from torchmetrics.functional.classification import auroc
import torch.nn.functional as F
import numpy as np
from ToxicityDataModule import ToxicityDataModule, max_token_len
from ToxicityClassifier import ToxicityClassifier
import json

In [2]:
data_path = "./dataset/processed/train.csv"
ds = pd.read_csv(data_path, dtype={"id": str})
ds.head

<bound method NDFrame.head of          Unnamed: 0                id  \
0                 0             59848   
1                 1             59849   
2                 2             59852   
3                 3             59855   
4                 4             59856   
...             ...               ...   
1964286     1964289  ffe987279560d7ff   
1964287     1964290  ffea4adeee384e90   
1964288     1964291  ffee36eab5c267c9   
1964289     1964292  fff125370e4aaaf3   
1964290     1964293  fff46fc426af1f9a   

                                              comment_text  score  
0        this is so cool. it is like, 'would you want y...      0  
1        thank you!! this would make my life a lot less...      0  
2        this is such an urgent design problem; kudos t...      0  
3        is this something i will be able to install on...      0  
4                     haha you guys are a bunch of losers.      9  
...                                                    ...    ...  
1

In [3]:
X = ds["comment_text"]
y = ds["score"]
X, _, y, _ = train_test_split(X, y, test_size=0.7, random_state=103)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=103)
train_ds = pd.DataFrame({"comment_text": X_train, "score": y_train})
val_ds = pd.DataFrame({"comment_text": X_test, "score": y_test})

val_ds.head

<bound method NDFrame.head of                                               comment_text  score
206654   so pat, why do you believe the oinker in a pan...      4
1652250  so are you saying that paul is simply a christ...      0
1164078  "for men, sex is a purely selfish act." \n\nth...      0
1378676  the much desired cfl diversity is strength t-s...      0
757356   it would appear as if all canadians are equal,...      2
...                                                    ...    ...
1667903  i agree cheetolini and his followers should be...      0
1256524  at least he has balls!  obama's wife had bigge...      8
1664258  did tulo hire a pi with his 250net weekly payc...      0
511669   do you really think their message will be hear...      0
1281987  when i was teaching school, i did science expe...      0

[117858 rows x 2 columns]>

In [4]:
config = json.load(open("config.json"))

data_module = ToxicityDataModule(train_ds, val_ds, max_token_len, model_name = config["model_name"], batch_size = config["batch_size"])
data_module.setup()

In [5]:
trainer = pl.Trainer(max_epochs=config["n_epochs"], num_sanity_val_steps=2, logger = True, enable_progress_bar = True, num_nodes = 1)
model = ToxicityClassifier(config)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
torch.set_float32_matmul_precision("medium")

model.load_state_dict(torch.load("./trained/v1.pth"))

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


<All keys matched successfully>

In [6]:
def predict_toxicity(model, data_module):
    predictions = trainer.predict(model, datamodule=data_module)
    flattened_predictions = np.stack([torch.sigmoid(torch.Tensor(p)) for batch in predictions for p in batch])
    return flattened_predictions

In [ ]:
predictions = predict_toxicity(model, data_module)
predictions

Predicting: |                                                                                                 …

In [ ]:
predictions = np.round(predictions)
predictions

In [ ]:
from sklearn import metrics
plt.figure(figsize=(15, 8))
for i, attribute in enumerate(attributes):
  fpr, tpr, _ = metrics.roc_curve(
      true_labels[:,i].astype(int), predictions[:, i])
  auc = metrics.roc_auc_score(
      true_labels[:,i].astype(int), predictions[:, i])
  plt.plot(fpr, tpr, label='%s %g' % (attribute, auc))
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc='lower right')
plt.title('RoBERTa Trained on UCC Datatset - AUC ROC')